In [ ]:
#reference to https://github.com/ThaddeusSegura/BERT_on_SQuAD/blob/master/SE_classification.ipynb

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, BertConfig, get_linear_schedule_with_warmup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm  # for showing progress bar
from datasets import load_dataset
import torch

device = torch.device('cuda:0')
#Using torch by GPU
if torch.cuda.is_available():
    device = torch.device('cuda:0')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')

In [ ]:

devset = pd.read_csv("dev-classifier-v2.0.csv")


model = BertForSequenceClassification.from_pretrained(
    'bert_qa_classifier_pt_3/', # 12-layer BERT
    num_labels = 2, #0:false 1:true
    output_attentions = False, # no attention output
    output_hidden_states = False, # no need for classifier
)

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
model.to(device)    #put model to device

In [ ]:
input_ids_list = []
attention_mask_list = []

labels = devset.answerable.values

for questions, context in zip(devset['question'], devset['context']):

    tokenized = tokenizer.encode_plus(questions,context,
                            add_special_tokens=True,    # Add `[CLS]` and `[SEP]`
                            truncation=True,
                            return_attention_mask=True,  # Construct attn. masks.
                            padding='max_length',       #512
                            max_length=512)
    
    input_ids_list.append(tokenized['input_ids'])
    attention_mask_list.append(tokenized['attention_mask'])

In [ ]:
input_ids = torch.tensor(input_ids_list)
attention_masks = torch.tensor(attention_mask_list)
labels = torch.tensor(labels)

data_3_elements = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)

batch_size = 32

dataloader = DataLoader(
            data_3_elements,  # The training samples.
            sampler = RandomSampler(data_3_elements), # Select batches randomly
            batch_size = batch_size)

In [ ]:
# Function to calculate the accuracy for predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
#eval loop
from tqdm.auto import tqdm
model.eval()

total_accuracy = 0
total_loss = 0

loop = tqdm(dataloader)
for batch in loop:
    batch_ids = batch[0].to(device)
    batch_mask = batch[1].to(device)
    batch_label = batch[2].to(device)
    
    
    with torch.no_grad(): 

        output = model(input_ids=batch_ids, 
                                token_type_ids=None, 
                                attention_mask=batch_mask, 
                                labels=batch_label)

    #get predict logits and move it with true label to cpu
    logits = output['logits']

    logits = logits.detach().cpu().numpy()
    true_labels = batch_label.to('cpu').numpy()

    total_accuracy += flat_accuracy(logits, true_labels)
    

accuracy = total_accuracy / len(dataloader)
print("Accuracy on dev set: " + str(accuracy))


In [ ]:
print("Accuracy on dev set: " + str(accuracy))